## Download CHL Data

In [1]:
# !mkdir -p chl_data

# for y in range(1997, 2000):
#     !wget -e robots=off -nd -r -np -l1 \
#           -A "S{y}*_chl_comp.hdf" \
#           https://spg-satdata.ucsd.edu/{y}/S{y}_chl_day/ \
#           -P chl_data

--2026-02-04 14:12:25--  https://spg-satdata.ucsd.edu/1997/S1997_chl_day/
Resolving spg-satdata.ucsd.edu (spg-satdata.ucsd.edu)... 169.228.224.44
Connecting to spg-satdata.ucsd.edu (spg-satdata.ucsd.edu)|169.228.224.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘chl_data/index.html.tmp’

index.html.tmp          [ <=>                ]  44.59K  --.-KB/s    in 0.001s  

2026-02-04 14:12:25 (58.1 MB/s) - ‘chl_data/index.html.tmp’ saved [45661]

Removing chl_data/index.html.tmp since it should be rejected.

--2026-02-04 14:12:25--  https://spg-satdata.ucsd.edu/1997/S1997_chl_day/?C=N;O=D
Reusing existing connection to spg-satdata.ucsd.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘chl_data/index.html?C=N;O=D.tmp’

index.html?C=N;O=D.     [ <=>                ]  44.59K  --.-KB/s    in 0s      

2026-02-04 14:12:25 (150 MB/s) - ‘chl_data/index.html?C=N;O=D.tmp’ saved [45661]


In [5]:
import os
import requests
from tqdm import tqdm  # for progress bar

years = [1998, 1999]
save_dir = "chl_data"
os.makedirs(save_dir, exist_ok=True)

for year in years:
    print(year)
    for day in tqdm(range(1, 366)):  # adjust to 366 if leap year
        day_str = f"{day:03d}"
        url = f"https://spg-satdata.ucsd.edu/{year}/S{year}_chl_day/S{year}{day_str}_chl_mapped.hdf"
        local_path = os.path.join(save_dir, f"S{year}{day_str}_chl_mapped.hdf")
        
        if os.path.exists(local_path):
            continue  # skip if already downloaded
        
        try:
            r = requests.get(url, timeout=30)
            if r.status_code == 200:
                with open(local_path, "wb") as f:
                    f.write(r.content)
            else:
                print(f"File not found: {url}")
        except Exception as e:
            print(f"Error downloading {url}: {e}")

1998


100%|███████████████████████████████████████| 365/365 [00:00<00:00, 3476.81it/s]


File not found: https://spg-satdata.ucsd.edu/1998/S1998_chl_day/S1998191_chl_mapped.hdf
File not found: https://spg-satdata.ucsd.edu/1998/S1998_chl_day/S1998320_chl_mapped.hdf
File not found: https://spg-satdata.ucsd.edu/1998/S1998_chl_day/S1998321_chl_mapped.hdf
File not found: https://spg-satdata.ucsd.edu/1998/S1998_chl_day/S1998322_chl_mapped.hdf
File not found: https://spg-satdata.ucsd.edu/1998/S1998_chl_day/S1998323_chl_mapped.hdf
File not found: https://spg-satdata.ucsd.edu/1998/S1998_chl_day/S1998324_chl_mapped.hdf
File not found: https://spg-satdata.ucsd.edu/1998/S1998_chl_day/S1998351_chl_mapped.hdf
1999


  7%|███                                       | 27/365 [00:02<00:22, 14.73it/s]

File not found: https://spg-satdata.ucsd.edu/1999/S1999_chl_day/S1999024_chl_mapped.hdf
File not found: https://spg-satdata.ucsd.edu/1999/S1999_chl_day/S1999025_chl_mapped.hdf


...

 27%|███████████▍                              | 99/365 [00:08<00:21, 12.24it/s]

File not found: https://spg-satdata.ucsd.edu/1999/S1999_chl_day/S1999097_chl_mapped.hdf


 88%|████████████████████████████████████▎    | 323/365 [00:27<00:02, 14.48it/s]

File not found: https://spg-satdata.ucsd.edu/1999/S1999_chl_day/S1999320_chl_mapped.hdf
File not found: https://spg-satdata.ucsd.edu/1999/S1999_chl_day/S1999321_chl_mapped.hdf
File not found: https://spg-satdata.ucsd.edu/1999/S1999_chl_day/S1999322_chl_mapped.hdf


100%|█████████████████████████████████████████| 365/365 [00:30<00:00, 11.80it/s]


In [ ]:
for y in range(2000, 2008):
    !wget -e robots=off -nd -r -np -l1 \
          -A "C{y}*_chl_comp.hdf" \
          https://spg-satdata.ucsd.edu/{y}/C{y}_chl_day/ \
          -P chl_data

In [ ]:
from pyhdf.SD import SD, SDC

# Example file
file_path = "/Users/deliacarpenter/Desktop/Research/primary-productivity/data/chl_data/C2000001_chl_comp.hdf"

# Open the HDF file
hdf = SD(file_path, SDC.READ)

# List all datasets
print("Datasets in HDF file:", hdf.datasets())

In [ ]:
chl_data = hdf.select('chl_2000001')[:]  # shape (rows, cols)
print("Shape of CHL array:", chl_data.shape)
print("Min/max CHL values:", chl_data.min(), chl_data.max())

In [ ]:
sds = hdf.select('chl_2000001')
print(sds.attributes())

In [ ]:
hdf.attributes()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyhdf.SD import SD, SDC

# --- Open your HDF file ---
hdf_file = "/Users/deliacarpenter/Desktop/Research/primary-productivity/data/chl_data/C2000001_chl_comp.hdf"
hdf = SD(hdf_file, SDC.READ)

# --- Select dataset ---
sds_name = list(hdf.datasets().keys())[0]  # 'chl_1997335_1997365'
sds = hdf.select(sds_name)
chl_data = sds[:]  # shape (3405, 3840)

# --- Handle fill values ---
fill_value = sds.attributes()['_FillValue']
chl_data = chl_data.astype(np.float32)
chl_data[chl_data == fill_value] = np.nan  # mask missing values

# --- Apply scaling (from attributes) ---
slope = sds.attributes()['Slope']
intercept = sds.attributes()['Intercept']
base = sds.attributes()['Base']

# Original equation: Base ** (Slope * data + Intercept)
chl_data_scaled = base ** (slope * chl_data + intercept)

# --- Approximate lat/lon grid ---
nrows, ncols = chl_data.shape
min_lat, max_lat = 16, 45
min_lon, max_lon = -135, -100

lats = np.linspace(max_lat, min_lat, nrows)
lons = np.linspace(min_lon, max_lon, ncols)
lon_grid, lat_grid = np.meshgrid(lons, lats)

# --- Plot ---
plt.figure(figsize=(12,6))
plt.pcolormesh(lon_grid, lat_grid, chl_data_scaled, shading='auto', cmap='viridis')
plt.colorbar(label='Chlorophyll (mg m^-3)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('SeaWiFS CHL Composite 2000 001')
plt.show()